In [2]:
import pandas as pd 
import numpy as np
import re
import csv
import json
from time import sleep
from bs4 import BeautifulSoup
import requests

In [ ]:
# https://www.youtube.com/watch?v=nZtzNTiBejM Webscraping in Python VOD

In [6]:
def extract_salary_info(job_title, state) :
    template = "https://www.salary.com/research/salary/listing/{}-salary/{}"
    
    url = template.format(job_title, state)
    
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None
    except requests.exceptions.ConnectionError:
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")
    pattern = re.compile(r'Occupation')
    script = soup.find("script", {"type" : "application/ld+json"}, text = pattern)
    
    json_raw = script.contents[0]
    json_data = json.loads(json_raw)
    
    job_title = json_data["name"]
    location = json_data["occupationLocation"][0]["name"]

    pct_10 = json_data["estimatedSalary"][0]["percentile10"]
    pct_25= json_data["estimatedSalary"][0]["percentile25"]
    pct_median = json_data["estimatedSalary"][0]["median"]
    pct_75 = json_data["estimatedSalary"][0]["percentile75"]
    pct_90= json_data["estimatedSalary"][0]["percentile90"]

    salary_data = (job_title, location, pct_10, pct_25,pct_median,pct_75,pct_90)
    
    return salary_data
    

In [39]:
state_abbr = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

state_abbr = pd.DataFrame(state_abbr)
state_abbr.to_csv("us_states.csv", index = False)

In [41]:
def main(job_title):

    with open('us_states.csv', newline="") as f:
        reader = csv.reader(f)
        states = [state for row in reader for state in row]
        
    salary_data = []

    for state in states:
        result = extract_salary_info(job_title, state)
        if result:
            salary_data.append(result)
            sleep(0.5)
            
    with open("salary-results.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Title","Location","pct_10","pct_25", "pct_median", "pct_75","pct_90"])
        writer.writerow(salary_data)
        
    return salary_data
        

In [42]:
data_jobs = ["data-scientist", "data-analyst", "data-engineer", 'data-architect','machine-learning-research-scientist',
             'machine-learning-engineer', 'analyst-business-intelligence', 'quantitative-analyst']

In [43]:
all_dfs = []
for job in data_jobs:
    df = pd.DataFrame(main(job))
    all_dfs.append(df)
    
all_salaries = pd.concat(all_dfs)
# runtime 36m 34.4s

C:\Users\Nick Climaco\AppData\Local\Temp\ipykernel_28248\3217170142.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script = soup.find("script", {"type" : "application/ld+json"}, text = pattern)
C:\Users\Nick Climaco\AppData\Local\Temp\ipykernel_28248\3217170142.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script = soup.find("script", {"type" : "application/ld+json"}, text = pattern)


In [45]:

all_salaries[1].nunique()

50

In [46]:
data_jobs_salary_df = all_salaries.copy()
data_jobs_salary_df.columns = ["Title", "Location", "Pct_10","Pct_25","Median","Pct_75","PCt_90"]

In [48]:
data_jobs_salary_df[['Title', 'Location']] = data_jobs_salary_df[['Title', 'Location']].astype(str)
data_jobs_salary_df[['Pct_10', 'Pct_25', 'Median', 'Pct_75', 'PCt_90']] = data_jobs_salary_df[['Pct_10', 'Pct_25', 'Median', 'Pct_75', 'PCt_90']].astype(int)

In [49]:
def show_summary(df) :
    
    print(80 * "+")
    print(f"DIMENSIONS : ({df.shape[0]}, {df.shape[1]})")
    
    print(80 * "+")
    print("COULMNS: \n")
    print(df.columns.values)
    
    print(80 * "+")
    print("DATA INFO: \n")
    print(df.dtypes)
    
    print(80 * "+")
    print("MISSING VALUES: \n")
    print(df.isnull().sum())
    
    print(80 * "+")
    print("NUMBER OF UNIQUE VALUES: \n")
    print(df.nunique())
    
show_summary(data_jobs_salary_df)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
DIMENSIONS : (350, 7)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
COULMNS: 

['Title' 'Location' 'Pct_10' 'Pct_25' 'Median' 'Pct_75' 'PCt_90']
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
DATA INFO: 

Title       object
Location    object
Pct_10       int32
Pct_25       int32
Median       int32
Pct_75       int32
PCt_90       int32
dtype: object
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MISSING VALUES: 

Title       0
Location    0
Pct_10      0
Pct_25      0
Median      0
Pct_75      0
PCt_90      0
dtype: int64
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
NUMBER OF UNIQUE VALUES: 

Title         7
Location     50
Pct_10      350
Pct_25      350
Median      349
Pct_75      349
PCt_90      349
dtype: int64


In [52]:
data_jobs_salary_df.to_csv("data_jobs_salaries.csv", index = True)